In [1]:
import pandas as pd
import os
import glob
import re
import numpy as np

"""
This notebook salvages a mistake I made when saving the translated answers.
The problemn is that the answers were saved in the order of their finishing the request using multiprocessing, which did not allign with answer_id, so they were named incorrectly.

This notebook takes an incorrect answer id from a folder of translated answers. It then finds the link to the incorrect answer_id, 
searches for the correct answer_id with the same link in the correct df and renames the file with the correct answer_id and save it to another folder.
"""



incorrect = pd.read_csv('../data/raw/questions_with_filurls.csv')
incorrect['answer_id'] = incorrect['answer_id'].astype('Int64')
incorrect.head()

,id,typeid,kategoriid,statusid,offentlighedskode,titel,dato,modtagelsesdato,frigivelsesdato,paragraf,paragrafnummer,spørgsmålsordlyd,spørgsmålstitel,spørgsmålsid,procedurenummer,grundnotatstatus,dagsordenudgavenummer,opdateringsdato,answer_id,filurl
0,2,16,14,1,O,"Er ministeren enig i vurderingen af, at grønla...",2014-01-02T00:00:00,NaN,2014-01-02T11:36:24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-05-29T11:11:34.047,3,['https://www.ft.dk/samling/20131/spoergsmaal/...
1,4,16,14,1,O,"Hvad agter ministeren at foretage sig, så kost...",2014-01-02T00:00:00,NaN,2014-01-20T10:12:24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-05-29T11:11:32.563,5,['https://www.ft.dk/samling/20131/spoergsmaal/...
2,16,16,14,1,O,"Mener ministeren, at det er fair, at det offen...",2014-01-03T00:00:00,NaN,2014-01-20T10:14:03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-05-29T11:11:15.453,19,['https://www.ft.dk/samling/20131/spoergsmaal/...
3,18,16,14,1,O,"Mener ministeren, at det vil være en god ide a...",2014-01-03T00:00:00,NaN,2014-01-20T10:13:46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-05-29T11:11:30.75,17,['https://www.ft.dk/samling/20131/spoergsmaal/...
4,31,16,14,1,O,"Vil udenrigsministeren i lyset af, at danske m...",2014-01-03T00:00:00,NaN,2014-01-03T13:53:04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-05-29T11:11:26.343,62,['https://www.ft.dk/samling/20131/spoergsmaal/...


In [2]:
#Loading correct from pickle
correct = pd.read_pickle('../data/raw/questions_with_filurls_newest.pickle')
correct['answer_id'] = correct['answer_id'].astype('Int64')
correct.head()

,id,typeid,kategoriid,statusid,offentlighedskode,titel,dato,modtagelsesdato,frigivelsesdato,paragraf,paragrafnummer,spørgsmålsordlyd,spørgsmålstitel,spørgsmålsid,procedurenummer,grundnotatstatus,dagsordenudgavenummer,opdateringsdato,answer_id,filurl
0,2,16,14,1,O,"Er ministeren enig i vurderingen af, at grønla...",2014-01-02T00:00:00,NaN,2014-01-02T11:36:24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-05-29T11:11:34.047,3,[https://www.ft.dk/samling/20131/spoergsmaal/S...
1,4,16,14,1,O,"Hvad agter ministeren at foretage sig, så kost...",2014-01-02T00:00:00,NaN,2014-01-20T10:12:24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-05-29T11:11:32.563,5,[https://www.ft.dk/samling/20131/spoergsmaal/S...
2,16,16,14,1,O,"Mener ministeren, at det er fair, at det offen...",2014-01-03T00:00:00,NaN,2014-01-20T10:14:03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-05-29T11:11:15.453,19,[https://www.ft.dk/samling/20131/spoergsmaal/S...
3,18,16,14,1,O,"Mener ministeren, at det vil være en god ide a...",2014-01-03T00:00:00,NaN,2014-01-20T10:13:46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-05-29T11:11:30.75,17,[https://www.ft.dk/samling/20131/spoergsmaal/S...
4,31,16,14,1,O,"Vil udenrigsministeren i lyset af, at danske m...",2014-01-03T00:00:00,NaN,2014-01-03T13:53:04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-05-29T11:11:26.343,62,[https://www.ft.dk/samling/20131/spoergsmaal/S...


In [3]:
correct['filurl'][5]

['https://www.ft.dk/samling/20131/spoergsmaal/S729/svar/1107358/1327364.pdf',
 'https://www.ft.dk/samling/20131/spoergsmaal/S729/svar/1107358/1327363.pdf']

In [8]:
incorrect_folder = '../data/translated_answers_incorrect/'
correct_folder = '../data/translated_answers_correct/'
import shutil
from tqdm import tqdm

from collections import defaultdict
import ast

from tqdm import tqdm
import re
import pandas as pd
from collections import defaultdict
import ast



# Create a hashmap to store the filurls and answer_ids from the incorrect_df
filurl_dict = dict()
for i, row in correct.iterrows():
    #check if row firlurl is not NaN or None
    if row['filurl'] is not pd.isna(row['filurl']):
        #check if the filurl is a list
        if isinstance(row['filurl'], list):
            #if it is a list, add each filurl to the hashmap
            for filurl in row['filurl']:
                filurl_dict[filurl] = row['answer_id']
        else:
            #if it is not a list, add the filurl to the hashmap
            filurl_dict[row['filurl']] = row['answer_id']
            



In [9]:
#count unique values in the hashmap
len(set(filurl_dict.values()))

7387

In [6]:
incorrect['filurl'] = incorrect['filurl'].apply(lambda x: ast.literal_eval(x) if not pd.isna(x) else x)

In [7]:
incorrect.head()

,id,typeid,kategoriid,statusid,offentlighedskode,titel,dato,modtagelsesdato,frigivelsesdato,paragraf,paragrafnummer,spørgsmålsordlyd,spørgsmålstitel,spørgsmålsid,procedurenummer,grundnotatstatus,dagsordenudgavenummer,opdateringsdato,answer_id,filurl
0,2,16,14,1,O,"Er ministeren enig i vurderingen af, at grønla...",2014-01-02T00:00:00,NaN,2014-01-02T11:36:24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-05-29T11:11:34.047,3,[https://www.ft.dk/samling/20131/spoergsmaal/S...
1,4,16,14,1,O,"Hvad agter ministeren at foretage sig, så kost...",2014-01-02T00:00:00,NaN,2014-01-20T10:12:24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-05-29T11:11:32.563,5,[https://www.ft.dk/samling/20131/spoergsmaal/S...
2,16,16,14,1,O,"Mener ministeren, at det er fair, at det offen...",2014-01-03T00:00:00,NaN,2014-01-20T10:14:03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-05-29T11:11:15.453,19,[https://www.ft.dk/samling/20131/spoergsmaal/S...
3,18,16,14,1,O,"Mener ministeren, at det vil være en god ide a...",2014-01-03T00:00:00,NaN,2014-01-20T10:13:46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-05-29T11:11:30.75,17,[https://www.ft.dk/samling/20131/spoergsmaal/S...
4,31,16,14,1,O,"Vil udenrigsministeren i lyset af, at danske m...",2014-01-03T00:00:00,NaN,2014-01-03T13:53:04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-05-29T11:11:26.343,62,[https://www.ft.dk/samling/20131/spoergsmaal/S...


In [10]:
# We go through each row in the incorrect df and check if the filurl is in the hashmap
a = 0
b = 0
c = 0
for i, row in tqdm(incorrect.iterrows()):
    #check if row firlurl is not NaN or None
    if row['filurl'] is not pd.isna(row['filurl']):
        #check if the filurl is a list
        try:
            if isinstance(row['filurl'], list):
                #if it is a list, check if any of the filurls are in the hashmap
                for filurl in row['filurl']:
                    if filurl in filurl_dict:
                        incorrect.at[i, 'correct_answer_id'] = filurl_dict[filurl]
                        a += 1
            else:
                #if it is not a list, check if the filurl is in the hashmap
                if row['filurl'] in filurl_dict:
                    incorrect.at[i, 'correct_answer_id'] = filurl_dict[row['filurl']]
                    b += 1 
        except:
            c += 1
            pass

print(a, b, c)

53056it [00:02, 19541.25it/s]

8065 0 0


In [11]:
test = incorrect[incorrect['answer_id'] == 3]['correct_answer_id'].values[0]
test


36.0

In [13]:
# Iterate through each file in the incorrect folder
for filename in os.listdir(incorrect_folder):
    if filename.endswith('.txt'):
        # Get the answer_id from the filename
        answer_id = int(os.path.splitext(filename)[0])
        # Look up the correct answer_id from the hashmap
        #find the correct answer_id in the incorrect df
        try:
            correct_answer_id = int(incorrect[incorrect['answer_id'] == answer_id]['correct_answer_id'].values[0])
        except:
            pass
        if correct_answer_id:
            # Define the paths to the input and output files
            input_path = os.path.join(incorrect_folder, filename)
            output_path = os.path.join(correct_folder, f'{correct_answer_id}.txt')
            try:
                # Copy the file and rename it to have the correct answer_id
                shutil.copyfile(input_path, output_path)
                #print(f"Successfully renamed {input_path} to {output_path}")
            except Exception as e:
                # Print an error message if the file can't be renamed
                print(f"Error renaming {input_path} to {output_path}: {e}")
        else:
            # Print an error message if the answer_id is missing or incorrect
            print(f"Missing or incorrect correct_answer_id for {answer_id}")

In [24]:
#get row of df where answer_id is 917
incorrect[incorrect['correct_answer_id'] == 917]

,id,typeid,kategoriid,statusid,offentlighedskode,titel,dato,modtagelsesdato,frigivelsesdato,paragraf,...,spørgsmålsordlyd,spørgsmålstitel,spørgsmålsid,procedurenummer,grundnotatstatus,dagsordenudgavenummer,opdateringsdato,answer_id,filurl,correct_answer_id
324,916,16,14,1,O,Vil ministeren med henvisning til artikel i Fy...,2014-02-24T00:00:00,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2015-05-29T09:21:14.72,935,[https://www.ft.dk/samling/20131/spoergsmaal/S...,917.0
